In [24]:
import os
import numpy as np
import torch
import torch.nn as nn
from music21 import converter, instrument, note, chord
from torch.utils.data import Dataset, DataLoader
from data_preparation import extract_sub_folders, get_notes, prepare_sequences

In [25]:
source = ".\\raw_datasets\\"            # https://www.kaggle.com/datasets/soumikrakshit/classical-music-midi
path = ".\\datasets\\classical_music"
extract_sub_folders(source, path)

In [26]:
class MusicLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(MusicLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # We use embeddings to learn better : we are searching for relations between note
        self.embedding = nn.Embedding(input_size, hidden_size)

        # LSTM and fully connected layers
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):

        x = self.embedding(x)

        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))  #[batch_size, sequence_length, hidden_size]

        out = out[:, -1, :]  # We keep the last sequence

        out = self.fc(out)  # Apply the fully connected layer to predict the next note based on the sequence

        return out

In [27]:
def train_music_model(model, train_loader, device, epochs):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if epoch % 5 == 0:
            torch.save(model.state_dict(), f"models\\epoch_{epoch + 1}.pt")
            print(f"model saved at epoch {epoch + 1}")

        print(f'Epoch {epoch+1}, Average Loss: {total_loss/len(train_loader):.4f}')

    torch.save(model.state_dict(), f"models\\epoch_{epochs + 1}.pt")
    print(f"model saved at epoch {epochs + 1}")


In [29]:
def generate_music(model, device, notes, note_to_int, int_to_note, sequence_length=100, generate_length=100):

    model.eval()

    # Initial sequence is picked at random point in the notes
    start_index = np.random.randint(0, len(notes) - sequence_length)
    pattern = notes[start_index:start_index + sequence_length]

    # It's the beggining of our generated music
    generated_notes = list(pattern)

    with torch.no_grad():
        for _ in range(generate_length):
            # Context to tenseur and send to device :
            pattern_indices = torch.tensor([note_to_int[note] for note in pattern],
                                           dtype=torch.long).unsqueeze(0).to(device)

            # Get the prediction from the model
            prediction = model(pattern_indices)

            # Get the index of the note with the highest probability from the prediction
            predicted_index = torch.argmax(prediction, dim=1).item()

            # Get the associated note
            predicted_note = int_to_note[predicted_index]

            generated_notes.append(predicted_note)

            # Update the pattern with the new note
            pattern = pattern[1:] + [predicted_note]

    return generated_notes

In [30]:
def save_generated_music_to_midi(generated_notes, output_file='generated_music.mid'):
    # Create a music21 stream
    stream = converter.stream.Stream()

    for note_str in generated_notes:
        try:
            if '.' in note_str:  # It's a chord
                chord_notes = [note.Note(int(p)) for p in note_str.split('.') if p.isdigit()]
                c = chord.Chord(chord_notes)

                # Set duration based on suffix
                if '_short' in note_str:
                    c.duration.quarterLength = 0.5
                elif '_medium' in note_str:
                    c.duration.quarterLength = 1.0
                else:
                    c.duration.quarterLength = 2.0

                stream.append(c)
            else:  # It's a single note
                pitch = note_str.split('_')[0]
                if pitch.isdigit():
                    n = note.Note(int(pitch))  # Convert pitch to MIDI note
                else:
                    n = note.Note(pitch)

                # Set duration based on suffix
                if '_short' in note_str:
                    n.duration.quarterLength = 0.5
                elif '_medium' in note_str:
                    n.duration.quarterLength = 1.0
                else:
                    n.duration.quarterLength = 2.0

                stream.append(n)
        except Exception as e:
            print(f"Error converting {note_str}: {e}")

    # Save as MIDI
    stream.write('midi', fp=output_file)

In [31]:
def get_output_name(folder_name):
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    os.listdir(folder_name)
    i = 1
    for file in os.listdir(folder_name):
        if file.endswith('.mid'):
            i += 1
    final_name = f"{folder_name}\\music{i}.mid"
    return final_name

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

notes = get_notes(path)
n_vocab = len(set(notes))

sequence_length = 100
network_input, network_output, note_to_int, int_to_note = prepare_sequences(notes, n_vocab, sequence_length)

# Create dataset and dataloader
dataset = torch.utils.data.TensorDataset(network_input, network_output)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Model parameters
input_size = n_vocab
hidden_size = 256
output_size = n_vocab

# Initialize model
model = MusicLSTM(input_size, hidden_size, output_size)

# Train model
train_music_model(model, dataloader, epochs=100, device=device)

Using device: cuda


C:\Users\elise\Documents\GitHub\Music_generation_LSTM\.venv\Lib\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\elise\Documents\GitHub\Music_generation_LSTM\.venv\Lib\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\elise\Documents\GitHub\Music_generation_LSTM\.venv\Lib\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=8, channel=None, data=b'Copyright 2004 by Bernd Kr\xfcger.'>; getting generic Instrument
  warnings.warn(
C:\Users\elise\Documents\GitHub\

Epoch 1, Average Loss: 4.1997
Epoch 2, Average Loss: 3.6160
Epoch 3, Average Loss: 3.3300
Epoch 4, Average Loss: 3.1350
Epoch 5, Average Loss: 2.9962
Epoch 6, Average Loss: 2.8958
Epoch 7, Average Loss: 2.8224
Epoch 8, Average Loss: 2.7681
Epoch 9, Average Loss: 2.7264
Epoch 10, Average Loss: 2.6924
Epoch 11, Average Loss: 2.6671
Epoch 12, Average Loss: 2.6487
Epoch 13, Average Loss: 2.6346
Epoch 14, Average Loss: 2.6235
Epoch 15, Average Loss: 2.6146
Epoch 16, Average Loss: 2.6112
Epoch 17, Average Loss: 2.6054
Epoch 18, Average Loss: 2.6029
Epoch 19, Average Loss: 2.5981
Epoch 20, Average Loss: 2.5983
Epoch 21, Average Loss: 2.5999
Epoch 22, Average Loss: 2.6008
Epoch 23, Average Loss: 2.6004
Epoch 24, Average Loss: 2.6012
Epoch 25, Average Loss: 2.6093
Epoch 26, Average Loss: 2.6139
Epoch 27, Average Loss: 2.6165
Epoch 28, Average Loss: 2.6267
Epoch 29, Average Loss: 2.6299
Epoch 30, Average Loss: 2.6303
Epoch 31, Average Loss: 2.6371
Epoch 32, Average Loss: 2.6414
Epoch 33, Average

In [34]:
# Utilisation
generated_music = generate_music(model, device, notes, note_to_int, int_to_note)
output_name = get_output_name("generated_music")
save_generated_music_to_midi(generated_music, output_name)

Erreur lors de la conversion de 5.8.10_short: File not found or no such format found for: 5
Erreur lors de la conversion de 10_short: Cannot have octave given before pitch name in '10'.
Erreur lors de la conversion de 3.7.10_short: File not found or no such format found for: 3
Erreur lors de la conversion de 10_short: Cannot have octave given before pitch name in '10'.
Erreur lors de la conversion de 10.2.5_short: File not found or no such format found for: 10
Erreur lors de la conversion de 10_short: Cannot have octave given before pitch name in '10'.
Erreur lors de la conversion de 7.10_long: File not found or no such format found for: 7
Erreur lors de la conversion de 10_short: Cannot have octave given before pitch name in '10'.
Erreur lors de la conversion de 0_short: Cannot have octave given before pitch name in '0'.
Erreur lors de la conversion de 5.8.10_short: File not found or no such format found for: 5
Erreur lors de la conversion de 2_short: Cannot have octave given before p